In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [2]:
# Read the dataset
df = pd.read_excel(r"C:\Users\tech\custom-import_data.xlsx")

In [3]:
# Inpect the dataset
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77789 entries, 0 to 77788
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  77789 non-null  int64 
 1   Custom Office       77789 non-null  object
 2   Reg Number          77789 non-null  object
 3   Importer            77789 non-null  int64 
 4   HS Code             77789 non-null  int64 
 5   FOB Value (N)       77789 non-null  int64 
 6   CIF Value (N)       77789 non-null  int64 
 7   Total Tax(N)        77789 non-null  int64 
 8   Receipt Number      76493 non-null  object
 9   Receipt Date        76493 non-null  object
 10  Mass(KG)            77789 non-null  int64 
 11  Country  of Origin  77789 non-null  object
 12  Country  of Supply  76826 non-null  object
 13  Nbr Of Containers   77789 non-null  int64 
 14  Container Nbr       45436 non-null  object
 15  Container Size      45436 non-null  object
dtypes: int64(8), object(8)

In [7]:
# Count null and duplicate values
null_summary = df.isnull().sum()
duplicate_count = df.duplicated().sum()
print("Null values per column:\n", null_summary)
print("Total duplicate rows:", duplicate_count)

Null values per column:
 ID                        0
Custom Office             0
Reg Number                0
Importer                  0
HS Code                   0
FOB Value (N)             0
CIF Value (N)             0
Total Tax(N)              0
Receipt Number         1296
Receipt Date           1296
Mass(KG)                  0
Country  of Origin        0
Country  of Supply      963
Nbr Of Containers         0
Container Nbr         32353
Container Size        32353
dtype: int64
Total duplicate rows: 0


In [9]:
# Standardize columns
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
df.rename(columns={
    'country__of_origin': 'country_of_origin',
    'country__of_supply': 'country_of_supply',
    'fob_value_(n)': 'fob_value(n)',
    'cif_value_(n)': 'cif_value(n)'
}, inplace=True)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77789 entries, 0 to 77788
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 77789 non-null  int64 
 1   custom_office      77789 non-null  object
 2   reg_number         77789 non-null  object
 3   importer           77789 non-null  int64 
 4   hs_code            77789 non-null  int64 
 5   fob_value(n)       77789 non-null  int64 
 6   cif_value(n)       77789 non-null  int64 
 7   total_tax(n)       77789 non-null  int64 
 8   receipt_number     76493 non-null  object
 9   receipt_date       76493 non-null  object
 10  mass(kg)           77789 non-null  int64 
 11  country_of_origin  77789 non-null  object
 12  country_of_supply  76826 non-null  object
 13  nbr_of_containers  77789 non-null  int64 
 14  container_nbr      45436 non-null  object
 15  container_size     45436 non-null  object
dtypes: int64(8), object(8)
memory usage: 9.5

In [11]:
# Handle HS code inconsistences
df['hs_code'] = df['hs_code'].astype(str).str.strip().str.extract(r'(\d{2,10})')[0]
df['hs_code'] = df['hs_code'].str[:6]

In [13]:
# Assign category using first two digits
df['hs_prefix'] = df['hs_code'].str[:2]

hs_category_map = {
    '01': 'Live animals',
    '02': 'Meat and edible meat offal',
    '03': 'Fish and seafood',
    '04': 'Dairy products',
    '07': 'Vegetables',
    '08': 'Fruits and nuts',
    '10': 'Cereals',
    '11': 'Milling products',
    '12': 'Oil seeds and oleaginous fruits',
    '13': 'Lac, gums, resins',
    '14': 'Vegetable products n.e.s.',
    '15': 'Animal or vegetable fats',
    '16': 'Prepared meat and seafood',
    '17': 'Sugars and sugar confectionery',
    '18': 'Cocoa and preparations',
    '19': 'Cereal preparations',
    '20': 'Vegetable/fruit/nut preparations',
    '21': 'Misc edible preparations',
    '22': 'Beverages and spirits',
    '23': 'Food industry residues',
    '24': 'Tobacco',
    '27': 'Mineral fuels and oils',
    '28': 'Inorganic chemicals',
    '29': 'Organic chemicals',
    '30': 'Pharmaceuticals',
    '31': 'Fertilizers',
    '32': 'Tanning or dyeing extracts',
    '33': 'Essential oils and cosmetics',
    '34': 'Cleaning products',
    '35': 'Albuminoidal substances',
    '36': 'Explosives',
    '37': 'Photographic goods',
    '38': 'Misc chemical products',
    '39': 'Plastics',
    '40': 'Rubber',
    '41': 'Raw hides and skins',
    '42': 'Leather articles',
    '43': 'Furskins',
    '44': 'Wood and articles of wood',
    '45': 'Cork and articles of cork',
    '46': 'Basketware',
    '47': 'Pulp of wood',
    '48': 'Paper and paperboard',
    '49': 'Printed books',
    '61': 'Apparel (knitted)',
    '62': 'Apparel (non-knitted)',
    '63': 'Other textile articles',
    '64': 'Footwear',
    '65': 'Headgear',
    '66': 'Umbrellas and walking-sticks',
    '67': 'Feathers and artificial flowers',
    '68': 'Stone and cement',
    '69': 'Ceramic products',
    '70': 'Glass and glassware',
    '71': 'Precious stones',
    '72': 'Iron and steel',
    '73': 'Articles of iron or steel',
    '74': 'Copper and articles thereof',
    '75': 'Nickel',
    '76': 'Aluminum',
    '78': 'Lead',
    '79': 'Zinc',
    '80': 'Tin',
    '81': 'Other base metals',
    '82': 'Tools and cutlery',
    '83': 'Misc metal articles',
    '84': 'Machinery',
    '85': 'Electrical machinery',
    '86': 'Railway equipment',
    '87': 'Vehicles',
    '88': 'Aircraft',
    '89': 'Ships and boats',
    '90': 'Optical instruments',
    '91': 'Clocks and watches',
    '92': 'Musical instruments',
    '93': 'Arms and ammunition',
    '94': 'Furniture',
    '95': 'Toys and games',
    '96': 'Misc manufactured articles',
    '97': 'Works of art',
    '99': 'Special classifications'
}
df['hs_category'] = df['hs_prefix'].map(hs_category_map).fillna('Other')

In [15]:
print(df['hs_category'].unique())

['Oil seeds and oleaginous fruits' 'Explosives' 'Plastics' 'Rubber'
 'Photographic goods' 'Wood and articles of wood' 'Fertilizers'
 'Raw hides and skins' 'Sugars and sugar confectionery' 'Stone and cement'
 'Printed books' 'Headgear' 'Inorganic chemicals' 'Lead'
 'Organic chemicals' 'Animal or vegetable fats' 'Mineral fuels and oils'
 'Cocoa and preparations' 'Furskins' 'Articles of iron or steel' 'Other'
 'Milling products' 'Iron and steel' 'Leather articles'
 'Lac, gums, resins' 'Footwear' 'Prepared meat and seafood'
 'Albuminoidal substances' 'Misc chemical products' 'Pulp of wood'
 'Pharmaceuticals' 'Zinc' 'Misc edible preparations'
 'Essential oils and cosmetics' 'Basketware' 'Paper and paperboard'
 'Tobacco' 'Aluminum' 'Food industry residues' 'Cereal preparations'
 'Precious stones' 'Vegetable products n.e.s.' 'Beverages and spirits'
 'Furniture' 'Apparel (knitted)' 'Tanning or dyeing extracts'
 'Cork and articles of cork' 'Cleaning products' 'Cereals'
 'Railway equipment' 'Oth

In [17]:
print(df['receipt_date'].value_counts())

receipt_date
19/01/1866    520
28/06/1866    513
18/07/1866    511
21/07/1866    509
28/10/1866    481
             ... 
13/02/1868      1
28/03/1867      1
08/01/1868      1
26/09/1867      1
04/12/1867      1
Name: count, Length: 384, dtype: int64


In [19]:
# Fix the receipt_date year to 2024 and 2025
df['receipt_date'] = pd.to_datetime(df['receipt_date'], errors='coerce')
df['receipt_date'] = df['receipt_date'].apply(
    lambda d: d.replace(year={1866: 2022, 1867: 2023, 1868: 2024, 1869: 2025}.get(d.year, d.year)) if pd.notnull(d) else d
)
print(df['receipt_date'].dt.year.unique())

[2022.   nan 2023. 2024. 2025.]


In [21]:
# Convert numeric columns
num_cols = ['fob_value(n)', 'cif_value(n)', 'total_tax(n)', 'mass(kg)', 'nbr_of_containers']
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [23]:
# Normalize text columns
text_cols = ['custom_office', 'importer', 'country_of_origin', 'country_of_supply', 'container_nbr', 'container_size']
for col in text_cols:
    df[col] = df[col].astype(str).str.strip().str.lower().replace('nan', np.nan)

In [25]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77789 entries, 0 to 77788
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 77789 non-null  int64         
 1   custom_office      77789 non-null  object        
 2   reg_number         77789 non-null  object        
 3   importer           77789 non-null  object        
 4   hs_code            77789 non-null  object        
 5   fob_value(n)       77789 non-null  int64         
 6   cif_value(n)       77789 non-null  int64         
 7   total_tax(n)       77789 non-null  int64         
 8   receipt_number     76493 non-null  object        
 9   receipt_date       29102 non-null  datetime64[ns]
 10  mass(kg)           77789 non-null  int64         
 11  country_of_origin  77789 non-null  object        
 12  country_of_supply  76826 non-null  object        
 13  nbr_of_containers  77789 non-null  int64         
 14  contai

In [27]:
# Check the value count percentage of country of origin and country of supply
origin_counts = df['country_of_origin'].value_counts(normalize=True)
supply_counts = df['country_of_supply'].value_counts(normalize=True)
print(origin_counts.head(20))
print(supply_counts.head(20))

country_of_origin
china                       0.349947
united states               0.107856
germany                     0.068943
united kingdom              0.068840
netherlands                 0.056332
belgium                     0.038322
hong kong                   0.037987
india                       0.033128
united arab emirates        0.032395
italy                       0.029914
republic of south africa    0.022767
france                      0.022651
sweden                      0.016172
spain                       0.014436
korea, republic of          0.010696
lebanon                     0.008652
japan                       0.008112
turkey                      0.007019
malaysia                    0.005862
singapore                   0.005618
Name: proportion, dtype: float64
country_of_supply
china                       0.331086
united states               0.103337
united kingdom              0.069065
netherlands                 0.068714
germany                     0.068701
belgiu

In [29]:
# Form a comaprison dataframe, find the correlation and fill the missing values
comparison_df = pd.concat([origin_counts, supply_counts], axis=1, keys=['origin_pct', 'supply_pct'])
if comparison_df.corr().iloc[0, 1] > 0.9:
    # Row-wise fill
    df['country_of_origin'] = df['country_of_origin'].fillna(df['country_of_supply'])
    df['country_of_supply'] = df['country_of_supply'].fillna(df['country_of_origin'])


In [31]:
comparison_df.corr()

,origin_pct,supply_pct
origin_pct,1.000000,0.998777
supply_pct,0.998777,1.000000


In [33]:
# Fill the missing values in container number and conatiner size
df['container_nbr'] = df['container_nbr'].fillna('unknown')
df['container_size'] = df['container_size'].fillna('unknown')

In [35]:
# Standardize container size values
container_size_map = {
    '20gp': '20ft', '20dv': '20ft', '20dc': '20ft', '20ot': '20ft', '20vh': '20ft', '20pl': '20ft',
    '20sn': '20ft', '20fl': '20ft', '20ut': '20ft', '20hd': '20ft', '20ve': '20ft', '20hc': '20ft',
    '22g1': '20ft', '22g0': '20ft', '22u1': '20ft', '22p3': '20ft', '22p1': '20ft', '22r1': '20ft',
    '25g1': '20ft', '2040': '20ft', '2200': '20ft',

    '40gp': '40ft', '40dv': '40ft', '40hc': '40ft', '40ot': '40ft', '40fl': '40ft', '40bk': '40ft',
    '40pl': '40ft', '40vh': '40ft', '40ut': '40ft', '40ps': '40ft', '40hd': '40ft', '40dc': '40ft',
    '42g1': '40ft', '42u0': '40ft', '42u1': '40ft', '42g0': '40ft', '42p0': '40ft', '42p3': '40ft',
    '42p1': '40ft', '42u5': '40ft', '4300': '40ft',

    '45g0': '45ft', '45g1': '45ft', '45u1': '45ft', '45p3': '45ft', '45gp': '45ft', '45r1': '45ft',
    '44g1': '45ft', '49p0': '45ft',

    '60mf': '60ft', '10dv': '10ft', '0': 'unknown', 'ncvh': 'unknown', 'hncv': 'unknown', 'l5g1': 'unknown'
}
df['container_size'] = df['container_size'].map(container_size_map).fillna('unknown')

In [37]:
# Count null and duplicate values
null_summary = df.isnull().sum()
duplicate_count = df.duplicated().sum()
print("Null values per column:\n", null_summary)
print("Total duplicate rows:", duplicate_count)

Null values per column:
 id                       0
custom_office            0
reg_number               0
importer                 0
hs_code                  0
fob_value(n)             0
cif_value(n)             0
total_tax(n)             0
receipt_number        1296
receipt_date         48687
mass(kg)                 0
country_of_origin        0
country_of_supply        0
nbr_of_containers        0
container_nbr            0
container_size           0
hs_prefix                0
hs_category              0
dtype: int64
Total duplicate rows: 0


In [39]:
# Remove invalid or negative values
for col in ['fob_value(n)', 'cif_value(n)', 'total_tax(n)', 'mass(kg)']:
    df = df[df[col] >= 0]

In [41]:
# Fill missing receipt_date with mode
receipt_mode = df['receipt_date'].mode()[0]
df['receipt_date'] = df['receipt_date'].fillna(receipt_mode)

In [43]:
# Drop missing receipt_number values
df = df[df['receipt_number'].notna()]

In [45]:
# Import Volume and Value KPIs
total_fob = df['fob_value(n)'].sum()
total_cif = df['cif_value(n)'].sum()
avg_import_value = df['fob_value(n)'].mean()
top_countries_by_value = df.groupby('country_of_origin')['fob_value(n)'].sum().sort_values(ascending=False).head(10)
top_importers_by_mass = df.groupby('importer')['mass(kg)'].sum().sort_values(ascending=False).head(10)
avg_mass_per_txn = df['mass(kg)'].mean()
print('The top 10 countries by values: \n', top_countries_by_value)
print(top_importers_by_mass)
print(avg_mass_per_txn)

The top 10 countries by values: 
 country_of_origin
china                       667423362775
lebanon                     114343994910
italy                       110300513789
india                       104520195135
united kingdom               92498330110
korea, republic of           78740311239
united states                74733363981
sweden                       68901520503
republic of south africa     61894973861
hong kong                    39777956468
Name: fob_value(n), dtype: int64
importer
7013     77025030
11327    75610492
30571    68418605
61090    60939646
69009    36299428
96059    30278549
35437    29428192
37193    28928351
62948    25975092
28895    22505722
Name: mass(kg), dtype: int64
17391.139267645405


In [47]:
# Taxation and Revenue KPIs
total_tax = df['total_tax(n)'].sum()
avg_tax = df['total_tax(n)'].mean()
tax_to_value_ratio = total_tax / total_fob if total_fob != 0 else np.nan
top_tax_contributors = df.groupby('importer')['total_tax(n)'].sum().sort_values(ascending=False).head(10)
print(tax_to_value_ratio)
print(top_tax_contributors)

0.14768256064669102
importer
69009    29522113044
37193    24644875768
11327    16126674225
96059    16052822615
94060    11128119401
44077     8551489748
35437     7561276465
67324     7189087220
28895     7020546613
45690     4681998721
Name: total_tax(n), dtype: int64


In [49]:
# Logistics and Shipment KPIs 
total_shipments = df.shape[0]
avg_containers_per_importer = df.groupby('importer')['nbr_of_containers'].mean().sort_values(ascending=False)
most_common_container_size = df['container_size'].mode()[0]
import_weight_by_country = df.groupby('country_of_origin')['mass(kg)'].sum().sort_values(ascending=False).head(10)

In [51]:
# Compliance and Processing KPIs 
transactions_per_custom = df['custom_office'].value_counts()
most_frequent_hs_codes = df['hs_code'].value_counts().head(10)

In [53]:
# Define a high-risk country list manually
high_risk_countries = ['iran', 'north korea', 'syria', 'sudan']
high_risk_pct = df[df['country_of_origin'].isin(high_risk_countries)].shape[0] / df.shape[0] * 100

In [55]:
# Create receipt year column
df['receipt_year'] = df['receipt_date'].dt.year

In [57]:
# Extract receipt month name
df['receipt_month'] = df['receipt_date'].dt.month_name()
print(df['receipt_month'].unique())

['March' 'January' 'April' 'May' 'June' 'July' 'August' 'February'
 'September' 'October' 'November' 'December']


In [59]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 76493 entries, 0 to 77783
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 76493 non-null  int64         
 1   custom_office      76493 non-null  object        
 2   reg_number         76493 non-null  object        
 3   importer           76493 non-null  object        
 4   hs_code            76493 non-null  object        
 5   fob_value(n)       76493 non-null  int64         
 6   cif_value(n)       76493 non-null  int64         
 7   total_tax(n)       76493 non-null  int64         
 8   receipt_number     76493 non-null  object        
 9   receipt_date       76493 non-null  datetime64[ns]
 10  mass(kg)           76493 non-null  int64         
 11  country_of_origin  76493 non-null  object        
 12  country_of_supply  76493 non-null  object        
 13  nbr_of_containers  76493 non-null  int64         
 14  container_n

In [61]:
# Save cleaned dataset
df.to_csv('trade_customs_data_cleaned.csv', index=False)
print("Cleaned data saved to 'trade_customs_data_cleaned.csv'")

Cleaned data saved to 'trade_customs_data_cleaned.csv'


In [97]:
df['container_size'].value_counts()

container_size
unknown    36233
40ft       18877
45ft       11964
20ft        9414
60ft           4
10ft           1
Name: count, dtype: int64

In [75]:
import matplotlib.pyplot as plt
import seaborn as sns

In [95]:
print(df['nbr_of_containers'].sum()/ (df.shape[0]))

44.5182957917718


In [101]:
print(df['mass(kg)'].sum())

1330300416


In [93]:
# Filter for Raw hides and skins category
raw_hides_df = df[df['hs_category'] == 'Raw hides and skins']

# Ensure no division by zero
total_tax_raw = raw_hides_df['total_tax(n)'].sum()
total_cif_raw = raw_hides_df['cif_value(n)'].sum()

# Compute ratio
tax_to_cif_ratio_raw = total_tax_raw / total_cif_raw if total_cif_raw != 0 else np.nan

print(f"Tax-to-CIF ratio for Raw hides and skins: {tax_to_cif_ratio_raw:.4f}")


Tax-to-CIF ratio for Raw hides and skins: 0.0198
